In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker-nightly

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 62 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 0s (130 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


In [ ]:
import pandas as pd
import tensorflow as tf
from tflite_model_maker import model_spec, text_classifier
from sklearn.model_selection import train_test_split
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

## Loading Amazon Review Data

In [ ]:
txt = open("/content/drive/MyDrive/EY/test.ft.txt", "r").read()

```__label__1 : -ve  -> 0```

```__label__2 : +ve  -> 1```

In [ ]:
X, y = [], []
for i in txt.split("\n")[:-1]:
  split_i = i.split(" ", 1)
  if split_i[0] == "__label__1":
    X.append(0)
  else:
    X.append(1)
  y.append(split_i[1])

#######
# Shortening Data  
X, y = X[:5000], y[:5000]
#######
Data = pd.DataFrame([X,y]).T
Data.columns = ["y", "X"]

In [ ]:
len(X)

5000

In [ ]:
train_data, test = train_test_split(Data, test_size=0.2)

In [ ]:
train_data.to_csv("/content/drive/MyDrive/EY/train.csv")
test.to_csv("/content/drive/MyDrive/EY/test.csv")

## BERT Model

In [ ]:
mb_spec = model_spec.get('mobilebert_classifier')
# mb_spec = model_spec.get('average_word_vec')

In [ ]:
# ds = tf.data.Dataset.from_tensor_slices(train_data.to_dict(orient="list"))
train_data = DataLoader.from_csv(
      filename="/content/drive/MyDrive/EY/train.csv",
      text_column='X',
      label_column='y',
      model_spec=mb_spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename="/content/drive/MyDrive/EY/test.csv",
      text_column='X',
      label_column='y',
      model_spec=mb_spec,
      is_training=False)

In [ ]:
model = text_classifier.create(train_data, model_spec=mb_spec, epochs=8)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


83/83 [==============================] - 1842s 21s/step - loss: 1.0176 - test_accuracy: 0.5502
Epoch 2/2
83/83 [==============================] - 1795s 22s/step - loss: 0.3694 - test_accuracy: 0.8441
Epoch 3/3
83/83 [==============================] - 1833s 22s/step - loss: 0.1969 - test_accuracy: 0.9322
Epoch 4/4
83/83 [==============================] - 1836s 22s/step - loss: 0.1171 - test_accuracy: 0.9704
Epoch 5/5
83/83 [==============================] - 1802s 22s/step - loss: 0.0707 - test_accuracy: 0.9842
Epoch 6/6
83/83 [==============================] - 1837s 22s/step - loss: 0.0444 - test_accuracy: 0.9900
Epoch 7/7
83/83 [==============================] - 1816s 22s/step - loss: 0.0246 - test_accuracy: 0.9947
Epoch 8/8
83/83 [==============================] - 1823s 22s/step - loss: 0.0167 - test_accuracy: 0.9967


In [ ]:
# model.summary()

In [ ]:
loss, acc = model.evaluate(test_data)

32/32 [==============================] - 128s 4s/step - loss: 0.4697 - test_accuracy: 0.9040


In [ ]:
model.export("/content/drive/MyDrive/EY/Model_90", export_format=[ExportFormat.LABEL, ExportFormat.VOCAB, ExportFormat.SAVED_MODEL, ExportFormat.TFLITE])

INFO:tensorflow:Assets written to: /tmp/tmpaqnkkxgm/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpaqnkkxgm/saved_model/assets


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmpjly59l_m/vocab.txt.


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/EY/Model_90/saved_model/assets


INFO:tensorflow:Saved vocabulary in /content/drive/MyDrive/EY/Model_90/vocab.txt.


INFO:tensorflow:Saved vocabulary in /content/drive/MyDrive/EY/Model_90/vocab.txt.


INFO:tensorflow:Saving labels in /content/drive/MyDrive/EY/Model_90/labels.txt


INFO:tensorflow:Saving labels in /content/drive/MyDrive/EY/Model_90/labels.txt
